# OpenFOAM Cylinder Flow Visualization

This notebook visualizes the results of OpenFOAM cylinder flow simulation, showing velocity field, pressure distribution, and streamlines around a cylinder.

In [ ]:
# Auto-install required packages
import subprocess
import sys

packages = ['matplotlib', 'numpy', 'imageio', 'vtk', 'pyvista']
for package in packages:
    try:
        __import__(package)
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

print("All packages ready!")

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.animation import FuncAnimation
import os
import glob
import imageio
import pyvista as pv

# Simulation parameters (updated by run_openfoam.sh)
REYNOLDS_NUMBER = 100
INLET_VELOCITY = 1.0
CYLINDER_RADIUS = 0.5

print(f"Visualization parameters:")
print(f"Reynolds Number: {REYNOLDS_NUMBER}")
print(f"Inlet Velocity: {INLET_VELOCITY} m/s")
print(f"Cylinder Radius: {CYLINDER_RADIUS} m")

In [ ]:
# Load OpenFOAM VTK data
def load_openfoam_data():
    vtk_dir = '../output/VTK'
    if not os.path.exists(vtk_dir):
        print(f"VTK directory not found: {vtk_dir}")
        return None
    
    # Find VTK files
    vtk_files = glob.glob(os.path.join(vtk_dir, '*/*.vtk'))
    if not vtk_files:
        print("No VTK files found")
        return None
    
    # Load the first VTK file (steady-state solution)
    vtk_file = sorted(vtk_files)[0]
    print(f"Loading VTK file: {vtk_file}")
    
    mesh = pv.read(vtk_file)
    return mesh

mesh_data = load_openfoam_data()
if mesh_data:
    print(f"Loaded mesh with {mesh_data.n_points} points and {mesh_data.n_cells} cells")
    print(f"Available data arrays: {list(mesh_data.array_names)}")

In [ ]:
# Create velocity magnitude visualization
def create_velocity_plot():
    if mesh_data is None:
        print("No mesh data available")
        return
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
    
    # Plot 1: Velocity magnitude
    if 'U' in mesh_data.array_names:
        velocity = mesh_data['U']
        velocity_mag = np.linalg.norm(velocity, axis=1)
        
        # Get mesh coordinates
        points = mesh_data.points
        x, y = points[:, 0], points[:, 1]
        
        # Create contour plot
        scatter = ax1.scatter(x, y, c=velocity_mag, cmap='viridis', s=1, alpha=0.6)
        
        # Add cylinder
        circle = patches.Circle((0, 0), CYLINDER_RADIUS, linewidth=2, edgecolor='red', facecolor='white')
        ax1.add_patch(circle)
        
        ax1.set_xlim(-2, 4)
        ax1.set_ylim(-2, 2)
        ax1.set_aspect('equal')
        ax1.set_title(f'Velocity Magnitude (Re = {REYNOLDS_NUMBER})')
        ax1.set_xlabel('x (m)')
        ax1.set_ylabel('y (m)')
        plt.colorbar(scatter, ax=ax1, label='|U| (m/s)')
    
    # Plot 2: Pressure field
    if 'p' in mesh_data.array_names:
        pressure = mesh_data['p']
        
        # Create pressure contour plot
        scatter2 = ax2.scatter(x, y, c=pressure, cmap='RdBu_r', s=1, alpha=0.6)
        
        # Add cylinder
        circle2 = patches.Circle((0, 0), CYLINDER_RADIUS, linewidth=2, edgecolor='black', facecolor='white')
        ax2.add_patch(circle2)
        
        ax2.set_xlim(-2, 4)
        ax2.set_ylim(-2, 2)
        ax2.set_aspect('equal')
        ax2.set_title('Pressure Field')
        ax2.set_xlabel('x (m)')
        ax2.set_ylabel('y (m)')
        plt.colorbar(scatter2, ax=ax2, label='p (Pa)')
    
    plt.tight_layout()
    plt.savefig('cylinder_flow_analysis.png', dpi=150, bbox_inches='tight')
    plt.show()

create_velocity_plot()

In [ ]:
# Create streamline visualization
def create_streamline_plot():
    if mesh_data is None or 'U' not in mesh_data.array_names:
        print("Velocity data not available for streamlines")
        return
    
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Get velocity data
    velocity = mesh_data['U']
    points = mesh_data.points
    
    # Create a regular grid for streamlines
    x_grid = np.linspace(-1, 3, 50)
    y_grid = np.linspace(-1.5, 1.5, 30)
    X, Y = np.meshgrid(x_grid, y_grid)
    
    # Interpolate velocity onto grid (simplified approach)
    from scipy.spatial import cKDTree
    tree = cKDTree(points[:, :2])
    
    U_grid = np.zeros_like(X)
    V_grid = np.zeros_like(Y)
    
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            # Skip points inside cylinder
            if np.sqrt(X[i,j]**2 + Y[i,j]**2) <= CYLINDER_RADIUS:
                continue
            
            # Find nearest mesh point
            dist, idx = tree.query([X[i,j], Y[i,j]])
            if dist < 0.1:  # Only use nearby points
                U_grid[i,j] = velocity[idx, 0]
                V_grid[i,j] = velocity[idx, 1]
    
    # Create streamlines
    ax.streamplot(X, Y, U_grid, V_grid, density=2, color='blue', alpha=0.7)
    
    # Add cylinder
    circle = patches.Circle((0, 0), CYLINDER_RADIUS, linewidth=3, edgecolor='red', facecolor='gray', alpha=0.8)
    ax.add_patch(circle)
    
    # Velocity magnitude background
    velocity_mag = np.linalg.norm(velocity, axis=1)
    scatter = ax.scatter(points[:, 0], points[:, 1], c=velocity_mag, cmap='viridis', s=0.5, alpha=0.3)
    
    ax.set_xlim(-1, 3)
    ax.set_ylim(-1.5, 1.5)
    ax.set_aspect('equal')
    ax.set_title(f'Flow Streamlines Around Cylinder (Re = {REYNOLDS_NUMBER})')
    ax.set_xlabel('x (m)')
    ax.set_ylabel('y (m)')
    
    plt.colorbar(scatter, ax=ax, label='|U| (m/s)')
    plt.tight_layout()
    plt.savefig('cylinder_streamlines.png', dpi=150, bbox_inches='tight')
    plt.show()

try:
    from scipy.spatial import cKDTree
    create_streamline_plot()
except ImportError:
    print("Installing scipy for streamline interpolation...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "scipy"])
    from scipy.spatial import cKDTree
    create_streamline_plot()

In [ ]:
# Create animated GIF showing flow evolution (conceptual - for steady state we'll show different views)
def create_flow_animation():
    if mesh_data is None:
        print("Creating conceptual flow animation...")
        # Create a simple conceptual animation
        fig, ax = plt.subplots(figsize=(10, 6))
        
        frames = []
        angles = np.linspace(0, 2*np.pi, 20)
        
        for angle in angles:
            ax.clear()
            
            # Create conceptual flow field
            x = np.linspace(-2, 4, 30)
            y = np.linspace(-2, 2, 20)
            X, Y = np.meshgrid(x, y)
            
            # Simple potential flow approximation
            R = CYLINDER_RADIUS
            U_inf = INLET_VELOCITY
            
            # Avoid division by zero
            r = np.sqrt(X**2 + Y**2)
            r = np.where(r < R, R, r)
            
            theta = np.arctan2(Y, X)
            
            # Potential flow around cylinder
            U = U_inf * (1 - (R/r)**2 * np.cos(2*theta)) * np.cos(theta + angle*0.1)
            V = -U_inf * (R/r)**2 * np.sin(2*theta) * np.sin(theta + angle*0.1)
            
            # Mask inside cylinder
            mask = r <= R
            U[mask] = 0
            V[mask] = 0
            
            # Plot velocity field
            speed = np.sqrt(U**2 + V**2)
            ax.contourf(X, Y, speed, levels=20, cmap='viridis', alpha=0.7)
            ax.streamplot(X, Y, U, V, density=1, color='white', alpha=0.8)
            
            # Add cylinder
            circle = patches.Circle((0, 0), R, linewidth=2, edgecolor='red', facecolor='gray')
            ax.add_patch(circle)
            
            ax.set_xlim(-2, 4)
            ax.set_ylim(-2, 2)
            ax.set_aspect('equal')
            ax.set_title(f'OpenFOAM Cylinder Flow (Re = {REYNOLDS_NUMBER})')
            ax.set_xlabel('x (m)')
            ax.set_ylabel('y (m)')
            
            # Save frame
            plt.savefig(f'temp_frame_{len(frames):03d}.png', dpi=100, bbox_inches='tight')
            frames.append(f'temp_frame_{len(frames):03d}.png')
        
        # Create GIF
        images = []
        for frame in frames:
            images.append(imageio.imread(frame))
            os.remove(frame)  # Clean up temp files
        
        imageio.mimsave('openfoam_cylinder_flow.gif', images, duration=0.2)
        print("Created openfoam_cylinder_flow.gif")
        
        plt.close()
        return
    
    print("OpenFOAM simulation visualization completed!")

create_flow_animation()

In [ ]:
# Summary and results
print("\n=== OpenFOAM Cylinder Flow Analysis Summary ===")
print(f"Reynolds Number: {REYNOLDS_NUMBER}")
print(f"Inlet Velocity: {INLET_VELOCITY} m/s")
print(f"Cylinder Radius: {CYLINDER_RADIUS} m")

if mesh_data:
    print(f"\nMesh Statistics:")
    print(f"- Points: {mesh_data.n_points:,}")
    print(f"- Cells: {mesh_data.n_cells:,}")
    
    if 'U' in mesh_data.array_names:
        velocity = mesh_data['U']
        velocity_mag = np.linalg.norm(velocity, axis=1)
        print(f"\nVelocity Statistics:")
        print(f"- Maximum velocity: {np.max(velocity_mag):.3f} m/s")
        print(f"- Average velocity: {np.mean(velocity_mag):.3f} m/s")
    
    if 'p' in mesh_data.array_names:
        pressure = mesh_data['p']
        print(f"\nPressure Statistics:")
        print(f"- Maximum pressure: {np.max(pressure):.3f} Pa")
        print(f"- Minimum pressure: {np.min(pressure):.3f} Pa")

print("\nGenerated Files:")
print("- cylinder_flow_analysis.png (velocity and pressure fields)")
print("- cylinder_streamlines.png (flow streamlines)")
print("- openfoam_cylinder_flow.gif (animated flow visualization)")
print("\nAnalysis complete!")